# Introductory Notebook 2: Mixed-Datatype Problems, Multi-Objective and Mixed-Objective Problems
This notebook will cover mixed datatypes and generating counterfactuals with multiple mixed objectives types.

In [1]:
import sys
sys.path.append('../src/')
import decode_mcd
import decode_mcd.multi_objective_problem as MOP
from decode_mcd import data_package


from decode_mcd import design_targets
from decode_mcd import counterfactuals_generator

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Creating a Dataset

First, lets create a dataset inspired by some basic arithmetic. This time, we will create a dataset with four variables of different types. `A` will be a random integer from 0 to 10 inclusive. `B` will be a random float from -1 to 1. `C` will be a random choice among "Add" "Subtract", "Multiply", and "Divide". Finally, `D` will be a boolean variable determining the sign of the expression. True will correspond to positive while False will mean negative. To effectively handle mixed datatypes will will use pandas.

In [2]:
num_data = 1000
A = np.random.randint(0, 10, num_data)
B = np.random.rand(num_data) * 2 - 1 #Randomized values which originally range from 0 to 1. Scales to -1 to 1.
C = np.random.choice(["Add", "Subtract", "Multiply", "Divide"], num_data)
D = np.random.choice([True, False], num_data)
x = pd.DataFrame({"A": A, "B": B, "C": C, "D": D})
display(x)


,A,B,C,D
0,6,-0.028449,Subtract,True
1,1,-0.926390,Subtract,True
2,8,-0.238767,Multiply,True
3,1,-0.368420,Multiply,False
4,9,0.713820,Add,True
...,...,...,...,...
995,3,0.243881,Divide,False
996,7,0.631665,Subtract,False
997,0,-0.329095,Add,False
998,3,0.767186,Add,False


We will create two functions. In the first one we will perform operation C(D(A), B). The second will perform D(C(B, A))>=0. For example, if C is add and D is False, function 1 will calculate -A+B while function 2 will check if -(B+A) is greater than 0 or not. We code up the `evaluate` function which takes in a dataframe with a set of `A`, `B`, `C`, and `D` values and returns a set of `O1` and `O2` values, which is what we call our function

In [3]:
def apply_operation(C, x, y):
    #Vectorized function to calculate C(x, y), for example if C is "Subtract", calculates x-y.
    add_mask = (C == "Add")
    subtract_mask = (C == "Subtract")
    multiply_mask = (C == "Multiply")
    divide_mask = (C == "Divide")
    result = np.zeros(len(C))
    result[add_mask] = x[add_mask] + y[add_mask]
    result[subtract_mask] = x[subtract_mask] - y[subtract_mask]
    result[multiply_mask] = x[multiply_mask] * y[multiply_mask]
    result[divide_mask] = x[divide_mask] / y[divide_mask]
    return result

def apply_inverse(D, x):
    #Vectorized function to calculate D(x), for example if D is False, returns -x. 
    return x*D-x*~D

def evaluate(x):
    #Evaluation function to calculate both objectives. x is an nx4 dataframe. 
    A = x["A"] #First isolate the individual variables from the provided dataframe
    B = x["B"]
    C = x["C"]
    D = x["D"].astype(bool)
    objective_1 = apply_operation(C, apply_inverse(D, A), B) #Calculate objective 1
    objective_2 = np.greater(apply_inverse(D, apply_operation(C, B, A)), 0) #Calculate objective 2
    return pd.DataFrame({"O1": objective_1, "O2": objective_2}) #Create a nx2 dataframe with the objective values
 

Let's evaluate our dataset.

In [4]:
y = evaluate(x)
display(y)

,O1,O2
0,6.028449,False
1,1.926390,False
2,-1.910135,False
3,0.368420,True
4,9.713820,True
...,...,...
995,-12.301072,False
996,-7.631665,True
997,-0.329095,True
998,-2.232814,False


Finally, we create our query. 

In [5]:
x_query = pd.DataFrame({"A": [0], "B": [0.0], "C": ["Add"], "D": [True]}, index = ["Query"])
display(x_query)

,A,B,C,D
Query,0,0.0,Add,True


### Setting up MCD

Now we are ready to set up the DataPackage. We specify the datatypes of our dataset. This time we have one of each type of variable. We specify the boundaries and options as specified earlier in the notebook.

In [6]:
from pymoo.core.variable import Real, Integer, Choice, Binary 
datatypes=[Integer(bounds=(0, 10)), 
           Real(bounds=(-1, 1)), 
           Choice(options=["Add", "Subtract", "Multiply", "Divide"]), 
           Binary()]

Finally, we create the `DataPackage`.

In [14]:
data = data_package.DataPackage(x=x, y=y, x_datatypes=datatypes)

Next, we create the design targets and the `MultiObjectiveProblem`. In this case, we create two design targets. We have one continuous target and one categorical target. Continuous targets should be used for any objective with ordinal significance (such as floats or ints), while categorical targets should be used for those without (such as classes or bools). We want set a hard lower bound of 10 for O1 with no upper bound, meaning C(D(A), B)>=10. We also set a requirement that O2 must be true, meaning that D(C(B, A))>=0

In [8]:
#We first set up a ContinuousTarget for O1, setting a minimum of 10 and setting no upper bound (i.e. infinity))
target_1 = design_targets.ContinuousTarget(label = "O1", lower_bound=10, upper_bound=np.inf)

#We then set up a CategoricalTarget for O2 specifying only True as the desired class. 
#Desired_classes is a list. In problems with multiple classes, this list specifies the acceptable classes.
target_2 = design_targets.CategoricalTarget(label = "O2", desired_classes=[True])

#We then create a DesignTargets object with the two targets.
y_targets = design_targets.DesignTargets(continuous_targets=[target_1], categorical_targets=[target_2])

In [9]:
problem = MOP.MultiObjectiveProblem(data_package=data, x_query = x_query, y_targets = y_targets, prediction_function=evaluate)

Finally, we create the `CounterfactualsGenerator`:

In [10]:
generator = counterfactuals_generator.CounterfactualsGenerator(problem=problem, pop_size=100, initialize_from_dataset=True)

### Generating Counterfactuals
Finally, we run the generator and sample.

In [11]:
generator.generate(n_generations=10)

1000 dataset entries found matching problem parameters
Initial population initialized from dataset of 1000 samples!
Training GA from 0 to 10 generations!
n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |        0 |      4 |  0.000000E+00 |  1.822443E+01 |             - |             -
     2 |      100 |      4 |  0.000000E+00 |  0.6343020102 |  0.000000E+00 |             f


c:\Users\Lyler\Documents\Multiobjective-Counterfactuals-for-Design\Getting Started\../src\decode_mcd\multi_objective_problem.py:284: RuntimeWarning: invalid value encountered in subtract
  satisfaction = np.maximum(actual - query_ub, query_lb - actual)
c:\Users\Lyler\Documents\Multiobjective-Counterfactuals-for-Design\Getting Started\../src\decode_mcd\multi_objective_problem.py:284: RuntimeWarning: invalid value encountered in subtract
  satisfaction = np.maximum(actual - query_ub, query_lb - actual)
c:\Users\Lyler\Documents\Multiobjective-Counterfactuals-for-Design\Getting Started\../src\decode_mcd\multi_objective_problem.py:284: RuntimeWarning: invalid value encountered in subtract
  satisfaction = np.maximum(actual - query_ub, query_lb - actual)
c:\Users\Lyler\Documents\Multiobjective-Counterfactuals-for-Design\Getting Started\../src\decode_mcd\multi_objective_problem.py:284: RuntimeWarning: invalid value encountered in subtract
  satisfaction = np.maximum(actual - query_ub, query_l

     3 |      200 |      5 |  0.000000E+00 |  0.3383962624 |  1.0000000000 |         ideal
     4 |      300 |      6 |  0.000000E+00 |  0.0673298138 |  0.0508201211 |             f
     5 |      400 |     10 |  0.000000E+00 |  0.000000E+00 |  0.0280032892 |         ideal
     6 |      500 |     11 |  0.000000E+00 |  0.000000E+00 |  0.5000000000 |         ideal
     7 |      600 |     11 |  0.000000E+00 |  0.000000E+00 |  0.0072845539 |             f
     8 |      700 |     11 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     9 |      800 |     11 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
    10 |      900 |     11 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f


c:\Users\Lyler\Documents\Multiobjective-Counterfactuals-for-Design\Getting Started\../src\decode_mcd\multi_objective_problem.py:284: RuntimeWarning: invalid value encountered in subtract
  satisfaction = np.maximum(actual - query_ub, query_lb - actual)


In [12]:
num_samples = 10 
counterfactuals = generator.sample(num_samples, include_dataset=False)
display(counterfactuals)

Scoring all counterfactual candidates!
Calculating diversity matrix!
Sampling diverse set of counterfactual candidates!
samples_index=[np.int64(103), np.int64(84), np.int64(74), np.int64(112), np.int64(30), np.int64(56), np.int64(86), np.int64(161), np.int64(59), np.int64(97)]
Done! Returning CFs


,A,B,C,D
0,10,0.000000,Add,True
1,1,0.034310,Divide,True
2,10,0.631092,Add,True
3,10,0.337746,Add,True
4,6,0.510026,Divide,True
5,10,0.444273,Add,True
6,10,0.741775,Add,True
7,3,0.146909,Divide,True
8,2,-0.051215,Divide,False
9,4,0.305240,Divide,True


Let's evaluate the counterfactuals we generated, we should see that every O1 value is greater or equal to 10, while every O2 value is true. 

In [13]:
evaluate(counterfactuals)

,O1,O2
0,10.000000,True
1,29.146443,True
2,10.631092,True
3,10.337746,True
4,11.764111,True
5,10.444273,True
6,10.741775,True
7,20.420853,True
8,39.051066,True
9,13.104445,True


### Adding Optimization Objectives to the Mix
Now let's try converting one of our two constraints into an objective. We will make our `ContinuousTarget` into a `MinimizationTarget`.  In this case, we do not specify bounds, but if we wanted to further contstraint the objective to minimize, this could be done with a `ContinuousTarget` and a `MinimizationTarget`. 

In [16]:
target_1 = design_targets.MinimizationTarget(label = "O1")
y_targets = design_targets.DesignTargets(minimization_targets=[target_1], categorical_targets=[target_2])
problem = MOP.MultiObjectiveProblem(data_package=data, x_query = x_query, y_targets = y_targets, prediction_function=evaluate)

In [17]:
generator = counterfactuals_generator.CounterfactualsGenerator(problem=problem, pop_size=100, initialize_from_dataset=True)
generator.generate(n_generations=10)
counterfactuals = generator.sample(num_samples, include_dataset=False)

1000 dataset entries found matching problem parameters
Initial population initialized from dataset of 1000 samples!
Training GA from 0 to 10 generations!
n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |        0 |     80 |  0.000000E+00 |  0.5604395604 |             - |             -
     2 |      100 |     90 |  0.000000E+00 |  0.000000E+00 |  0.0221306574 |         ideal
     3 |      200 |    100 |  0.000000E+00 |  0.000000E+00 |  0.0168149018 |         nadir
     4 |      300 |    100 |  0.000000E+00 |  0.000000E+00 |  0.0091250898 |         ideal
     5 |      400 |    100 |  0.000000E+00 |  0.000000E+00 |  0.0078025137 |             f
     6 |      500 |    100 |  0.000000E+00 |  0.000000E+00 |  0.0126242107 |             f
     7 |      600 |    100 |  0.000000E+00 |  0.000000E+00 |  0.0018557945 |             f
     8 |      700 |    100 |  0.000000E+00 |  0.000000E+00 |  0.0043779454 |             f
     9 |      800 |    100 

We can see that in this case, the generated counterfactuals have much smaller O1 values. 

In [18]:
evaluate(counterfactuals)

,O1,O2
0,-0.589413,True
1,-0.588235,True
2,-1.067241,True
3,-0.600161,True
4,-0.590719,True
5,-1.255780,True
6,-0.662299,True
7,-0.705470,True
8,-1.150928,True
9,-0.674789,True
